# Process : SOCAT
_process SOCAT product_

- **Original**: `/local/data/artemis/observations/SOCAT`
- **Processed**: `/local/data/artemis/observations/SOCAT`
- **Variable**: spco2
- **Temporal**: Monthly, 198112 - 201908
- **Spatial**: 1x1 degree
- **todo**: 

Laste update : 20 September 2019 L. Gloege

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
# Observations directory
dir_obs = '/local/data/artemis/observations'

In [3]:
# Load raw data
ds = xr.open_dataset(f'{dir_obs}/SOCAT/originals/SOCATv2019_tracks_gridded_monthly.nc')

In [4]:
# Roll so longitude is 0-360
ds_rolled = ds.roll(xlon=180, roll_coords='lon')

In [5]:
print('start:', str(ds_rolled['tmnth'][0].values))
print('end:', str(ds_rolled['tmnth'][-1].values))

start: 1970-01-16T12:00:00.000000000
end: 2018-12-16T12:00:00.000000000


In [6]:
# Create date vector
dates = pd.date_range(start='1970-01-01T00:00:00.000000000', 
                      end='2018-12-01T00:00:00.000000000',freq='MS')+ np.timedelta64(14, 'D')
da_dates = xr.DataArray(dates, dims='time')
np.shape(da_dates)

(588,)

# Put data into dataset

In [11]:
ds_out = xr.Dataset(
    {
    'spco2':(['time','lat', 'lon'], ds_rolled['fco2_ave_unwtd'], 
             {'long_name':'Arithmetic mean of all fco2 recomputed values found in the grid cell.',
               'units':'uatm'}),
    },

    coords={
    'lat': (['lat'], ds_rolled['ylat'], {'units':'degrees_north'}),
    'lon': (['lon'], np.arange(0.5, 360, 1), {'units':'degrees_east'}),
    'time': (['time'], da_dates)
    },
    attrs={'contact':'Luke Gloege (gloege@ldeo.columbia.edu)',
           'creation_date':'21 September 2019',
           'notes':'original data, trimmed 1982-2017',
           'original_data':'https://www.socat.info/'})

In [14]:
ds_out.info()

xarray.Dataset {
dimensions:
	lat = 180 ;
	lon = 360 ;
	time = 588 ;

variables:
	float32 spco2(time, lat, lon) ;
		spco2:long_name = Arithmetic mean of all fco2 recomputed values found in the grid cell. ;
		spco2:units = uatm ;
	float64 lat(lat) ;
		lat:units = degrees_north ;
	float64 lon(lon) ;
		lon:units = degrees_east ;
	datetime64[ns] time(time) ;

// global attributes:
	:contact = Luke Gloege (gloege@ldeo.columbia.edu) ;
	:creation_date = 21 September 2019 ;
	:notes = original data, trimmed 1982-2017 ;
	:original_data = https://www.socat.info/ ;
}

## Trim 1982-2018

In [15]:
ds_out.sel(time=slice('1982-01-01','2018-12-31'))

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360, time: 444)
Coordinates:
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * time     (time) datetime64[ns] 1982-01-15 1982-02-15 ... 2018-12-15
Data variables:
    spco2    (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan
Attributes:
    contact:        Luke Gloege (gloege@ldeo.columbia.edu)
    creation_date:  21 September 2019
    notes:          original data, trimmed 1982-2017
    original_data:  https://www.socat.info/

In [16]:
ds_out.info()

xarray.Dataset {
dimensions:
	lat = 180 ;
	lon = 360 ;
	time = 588 ;

variables:
	float32 spco2(time, lat, lon) ;
		spco2:long_name = Arithmetic mean of all fco2 recomputed values found in the grid cell. ;
		spco2:units = uatm ;
	float64 lat(lat) ;
		lat:units = degrees_north ;
	float64 lon(lon) ;
		lon:units = degrees_east ;
	datetime64[ns] time(time) ;

// global attributes:
	:contact = Luke Gloege (gloege@ldeo.columbia.edu) ;
	:creation_date = 21 September 2019 ;
	:notes = original data, trimmed 1982-2017 ;
	:original_data = https://www.socat.info/ ;
}

# Save as netcdf file

In [17]:
ds_out.to_netcdf(f'{dir_obs}/SOCAT/processed/spco2_1x1_mon_SOCATv2019_197001-201812.nc')